In [ ]:
#import xlrd
import json
import cv2
import os
from collections import Counter
from sklearn.cluster import KMeans
from matplotlib import colors
import matplotlib.pyplot as plt
import numpy as np

#customize data reading here
pth = "/UC San Diego/Sherry Jiang - Instagram Study Subject Data/"
pth2 = "/UC\ San\ Diego/Sherry\ Jiang\ -\ Instagram\ Study\ Subject\ Data/"
# bash command: ls -d */ | cut -f1 -d'/' > directories.txt
#sheet = xlrd.open_workbook(pth + 'quatrics_data.xlsx').sheet_by_name('Social+Media+and+Health_January')
#profile_id = [[sheet.cell_value(r, c) for c in range(sheet.ncols)] for r in range(sheet.nrows)]

with open('directories.txt') as f:
    lines = f.readlines()

checked=[]
"""for i in profile_id:
    if i not in lines:
        checked.append(i)
print(checked)"""
# captions, nltk -> polarity 
# image data -> hue, saturation

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
from pathlib import Path
from skimage import color, io
import pandas as pd

In [ ]:
def analyze(img):
    clf = KMeans(n_clusters = 5)
    color_labels = clf.fit_predict(img)
    center_colors = clf.cluster_centers_
    counts = Counter(color_labels)
    ordered_colors = [center_colors[i] for i in counts.keys()]
    hex_colors = [rgb_to_hex(ordered_colors[i]) for i in counts.keys()]

    plt.figure(figsize = (12, 8))
    plt.pie(counts.values(), labels = hex_colors, colors = hex_colors)

    plt.savefig("results/my_pie.png")
    print("Found the following colors:\n")
    for color in hex_colors:
      print(color)

def rgb_to_hex(rgb_color):
    hex_color = "#"
    for i in rgb_color:
        hex_color += ("{:02x}".format(int(i)))
    return hex_color

def preprocess(raw):
    image = cv2.resize(raw, (900, 600), interpolation = cv2.INTER_AREA)                                          
    image = image.reshape(image.shape[0]*image.shape[1], 3)
    return image

In [ ]:
##  get_hsv: converts image from rgb to hsv (Hue, Saturation, V=Brightness)
def get_hsv(path):
	if not os.path.exists(Path(path)):
		return 0
	im = io.imread(path)
	im = color.rgb2hsv(im) # http://scikit-image.org/docs/dev/api/skimage.color.html#rgb2hsv
	hsv = []
	for i in range(len(im.shape)):
		if len(im.shape) == 2: #b/w photo
			imdata = im[:,i]
		elif len(im.shape) == 3: #color photo
			imdata = im[:,:,i]
		avg = np.mean(imdata)
		hsv.append( avg )
	return hsv

In [ ]:
saved_data = []
num_img, num_posts = 0, 0
for i in lines:
    # emojis -> convert into more meaningful words?
    # loop through all profiles and get the relevant json file
    #p_img = pth2 + 'instagram_data/'+i[:-1]+"/media/posts/"
    p_img = pth + "instagram_data/" + i[:-1] + "/"
    p_text = pth + "instagram_data/" + i[:-1] + "/content/posts_1.json"
    if not os.path.exists(Path(p_text)):
        continue
    #f = open(p_text)
    with open(p_text, 'r') as f:
        content = f.read()
        print(i)
        data = json.loads(content)
    for d in data:
        if not isinstance(d, dict) or 'title' not in d.keys():
            continue
        title = d['title'].encode('latin1').decode('utf8')
        
        ps = sia.polarity_scores(title)
        hsv = [0, 0, 0]
        count = 0
        num_posts += 1
        for m in d['media']:
            if ".jpg" in m["uri"]:
                thsv = get_hsv(p_img + m["uri"])
                if thsv == 0:
                    continue
                hsv = [a + b for a, b in zip(hsv, thsv)]
                count += 1
                num_img += 1
        if count != 0:
            avg_hsv = [x / count for x in hsv]
        saved_data.append([i[:-1], d["creation_timestamp"], title, ps, avg_hsv[0], avg_hsv[1], avg_hsv[2]])
        # creation timestamp | text | polarity score | profile-id -> csv
        # hsv extraction -> output compare with current
        # analysis writeup -> methods used in analysis/feature extraction(text: captions, vader polarity score, number of files; image: hsv, number of images, normalization/mean)
    """cmd = "cd "+p_img + "; ls -d */ | cut -f1"
    dates = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
    dates = dates.communicate()[0].strip().decode("utf-8").split("\n")
    p_img = pth + 'instagram_data/'+i[:-1]+"/media/posts/"
    for d in dates:
        file_list=os.listdir(p_img+d[:-1])
        for f in file_list:
            hsv = get_hsv(p_img+d[:-1]+"/"+f)
            print(hsv)
            #image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            #modified_image = preprocess(image)
            #analyze(modified_image)"""

In [ ]:
import csv
fields = ['id', 'timestamp', 'caption', 'polarity score', 'hue', 'saturation', 'brightness']
with open("./instagram_posts_data.csv", 'w') as f:
    w = csv.writer(f)

    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(saved_data)
print(num_img, num_posts)
# print(captions[3].encode('latin1').decode('utf8'))
# cleaning: english language only; 
# image stats (hsv) -> logistic regression

In [ ]:
import pandas as pd
import sklearn

x = pd.read_csv("./instagram_posts_data.csv")
survey = pd.read_csv("./survey_with_scores.csv")
print(x.head())
print(y.head())

In [ ]:
y = survey[['ResponseId', 'UCLA_LS_mean','UCLA_lonely']]
color_X = x[['id', 'hue', 'saturation', 'brightness']]
color_X = color_X.groupby(['id'])['hue', 'saturation', 'brightness'].mean().reset_index()
color_X.head()

In [ ]:
m = y.ResponseId.isin(color_X.id)
y = y[m]
y = y.sort_values('ResponseId').reset_index()
color_X = color_X.sort_values('id').reset_index()
print(color_X.head())
print(y.head())
# most recent 3/6/12 months' posts -> number of posts -> number of pictures
# binary/multiclass/linear classification

In [ ]:
color_X = color_X.drop(index=[60], axis=0).reset_index()
color_X['UCLA_LS_mean'] = y['UCLA_LS_mean'].to_numpy()
color_X['UCLA_lonely'] = y['UCLA_lonely'].to_numpy()
print(color_X.id, y.ResponseId)

In [ ]:
color_X = color_X.dropna()
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(color_X[['hue', 'saturation', 'brightness']].values, color_X[['UCLA_lonely']].values, test_size=0.25, random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(x_train, y_train)
pred = clf.predict(x_test)
score = clf.score(x_test, y_test)
print(score)

In [ ]:
saved_data = []
num_comments, num_posts, num_following, num_follower = [], [], [], []
for i in lines:
    # emojis -> convert into more meaningful words?
    # loop through all profiles and get the relevant json file
    #p_img = pth2 + 'instagram_data/'+i[:-1]+"/media/posts/"
    p_comment = pth + "instagram_data/" + i[:-1] + "/comments/post_comments.json"
    p_profilephoto = pth + "instagram_data/" + i[:-1] + "/content/profile_photos.json"
    p_search = pth + "instagram_data/" + i[:-1] + "/recent_searches/word_or_phrases_searches.json"
    p_likedposts = pth + "instagram_data/" + i[:-1] + "/likes/liked_posts.json"
    p_likedcomments = pth + "instagram_data/" + i[:-1] + "/likes/liked_comments.json"
    p_followers = pth + "instagram_data/" + i[:-1] + "/followers_and_following/followers.json"
    p_following = pth + "instagram_data/" + i[:-1] + "/followers_and_following/following.json"
    p_priv = pth + "instagram_data/" + i[:-1] + "/login_and_account_creation/account_privacy_changes.json"

    if not os.path.exists(Path(p_profilephoto)):
        continue
    
    comments = []
    with open(p_comment, 'r') as f:
        content = f.read()
        print(i)
        data = json.loads(content)
    for d in data:
        if not isinstance(d, dict) or 'title' not in d.keys():
            continue
        title = d['title'].encode('latin1').decode('utf8')
        value = d['string_list_data'][0]['value'].encode('latin1').decode('utf8')
        
        #title = d['value'].encode('latin1').decode('utf8')
        l = [i, title, value, d['string_list_data'][0]['timestamp']]
        comments.append(l)
    
    searches = []
    with open(p_search, 'r') as f:
        content = f.read()
        data = json.loads(content)
    for d in data:
        if not isinstance(d, dict) or 'string_list_data' not in d.keys():
            continue
        l = [i, d['string_list_data'][0]['value'], d['string_list_data'][0]['timestamp']]
        searches.append(l)

    with open(p_likedposts, 'r') as f:
        content = f.read()
        data = json.loads(content)
    num_posts.append(len(data))


    with open(p_likedcomments, 'r') as f:
        content = f.read()
        data = json.loads(content)
    num_comments.append(len(data))

    with open(p_followers, 'r') as f:
        content = f.read()
        data = json.loads(content)
    num_follower.append(len(data))


    with open(p_following, 'r') as f:
        content = f.read()
        data = json.loads(content)
    num_following.append(len(data))

    privacy = []
    with open(p_priv, 'r') as f:
        content = f.read()
        data = json.loads(content)
    for d in data:
        if not isinstance(d, dict) or 'title' not in d.keys():
            continue
        l = [i, d['title'], d['string_list_data'][0]['timestamp']]
        privacy.append(l)

In [ ]:
saved_data = []
num_comments, num_posts, num_following, num_follower = [], [], [], []

for i in lines:
    p_comment = pth + "instagram_data/" + i[:-1] + "/comments/post_comments.json"

    if not os.path.exists(Path(p_comment)):
        continue
    
    comments = []
    with open(p_comment) as f:
        print(i)
        data = json.load(f)
    #f = open(p_comment)
    #print(type(f))
    #data = json.load(f)
    
    for d in data:
        print(d)
        if not isinstance(d, dict) or 'title' not in d.keys():
            continue
        #title = d['value'].encode('latin1').decode('utf8')
        print('jdiosjfai')
        l = [i, d['title'], d['string_list_data'][0]['value'], d['string_list_data'][0]['timestamp']]
        comments.append(l)
    
    #f.close()

In [ ]:
saved_data = []
num_comments, num_posts, num_following, num_follower = [], [], [], []
comments = []
for i in lines:
    p_comment = pth + "instagram_data/" + i[:-1] + "/comments/post_comments.json"

    if not os.path.exists(Path(p_comment)):
        continue
    
    
    with open(p_comment, 'r') as f:
        print(i)
        data = json.loads(f.read())
    data = pd.json_normalize(data, record_path =['comments_media_comments'])

    for ind, d in data.iterrows():
        #if not isinstance(d, dict) or 'title' not in d.keys():
        #    continue

        title = d['title'].encode('latin1').decode('utf8')
        value = d['string_list_data'][0]['value'].encode('latin1').decode('utf8')

        l = [i[:-1], title, value, d['string_list_data'][0]['timestamp']]
        comments.append(l)

In [ ]:
import csv
fields = ['id', 'commenter_username', 'comment', 'timestamp']
with open("./instagram_comments_data.csv", 'w') as f:
    w = csv.writer(f)

    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(comments)
print(len(comments))

In [ ]:
saved_data = []
num_comments, num_posts, num_following, num_follower = [], [], [], []
searches = []
for i in lines:
    p_search = pth + "instagram_data/" + i[:-1] + "/recent_searches/word_or_phrase_searches.json"

    if not os.path.exists(Path(p_search)):
        continue
    print(i)
    
    with open(p_search, 'r') as f:
        data = json.loads(f.read())
    data = pd.json_normalize(data, record_path =['searches_keyword'])
    
    print(data)
    for ind, d in data.iterrows():
            #if not isinstance(d, dict) or 'string_list_data' not in d.keys():
            #    continue
        l = [i[:-1], d['string_map_data.Search.value'], d['string_map_data.Time.timestamp']]
        searches.append(l)

In [ ]:
fields = ['id', 'search_value', 'timestamp']
with open("./instagram_searches_data.csv", 'w') as f:
    w = csv.writer(f)

    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(searches)
print(len(searches))

In [ ]:
saved_data = []
for i in lines:

    p_likedposts = pth + "instagram_data/" + i[:-1] + "/likes/liked_posts.json"
    p_likedcomments = pth + "instagram_data/" + i[:-1] + "/likes/liked_comments.json"
    p_followers = pth + "instagram_data/" + i[:-1] + "/followers_and_following/followers.json"
    p_following = pth + "instagram_data/" + i[:-1] + "/followers_and_following/following.json"
    p_priv = pth + "instagram_data/" + i[:-1] + "/login_and_account_creation/account_privacy_changes.json"
    p_acc = pth + "instagram_data/" + i[:-1] + "/account_information/personal_information.json"

    if not os.path.exists(Path(p_acc)):
        continue
    print(i)
    temp = [i[:-1]]
    if not os.path.exists(Path(p_likedposts)):
        temp.append(0)
    else:
        with open(p_likedposts, 'r') as f:
            data = json.loads(f.read())
        data = pd.json_normalize(data, record_path =['likes_media_likes'])
        temp.append(data.shape[0])

    if not os.path.exists(Path(p_likedcomments)):
        temp.append(0)
    else:
        with open(p_likedcomments, 'r') as f:
            data = json.loads(f.read())
        data = pd.json_normalize(data, record_path =['likes_comment_likes'])
        temp.append(data.shape[0])

    if not os.path.exists(Path(p_followers)):
        temp.append(0)
    else:
        with open(p_followers, 'r') as f:
            data = json.loads(f.read())
        data = pd.json_normalize(data, record_path =['relationships_followers'])
        temp.append(data.shape[0])

    if not os.path.exists(Path(p_following)):
        temp.append(0)
    else:
        with open(p_following, 'r') as f:
            data = json.loads(f.read())
        data = pd.json_normalize(data, record_path =['relationships_following'])
        temp.append(data.shape[0])

    with open(p_acc, 'r') as f:
        data = json.loads(f.read())
    data = pd.json_normalize(data, record_path =['profile_user'])
    temp.append(data.iloc[0][-1])

    temp.append([])
    temp.append([])
    if not os.path.exists(Path(p_priv)):
        continue

    with open(p_priv, 'r') as f:
        data = json.loads(f.read())
    data = pd.json_normalize(data, record_path =['account_history_account_privacy_history'])
    data = data.set_axis(['title', 'href', 'value', 'timestamp'], axis=1, inplace=False)
    df = data.groupby('title')['timestamp'].apply(list).reset_index(name='new')
    temp2 = df['new'].tolist()
    
    if "Switched to Public" not in pd.unique(data['title']) and "Switched to Private" in pd.unique(data['title']):
        temp[6] = temp2[0]
    elif "Switched to Private" in pd.unique(data['title']):
        temp[7] = temp2[1]
        if "Switched to Public" in pd.unique(data['title']):
            temp[6] = temp2[0]
    saved_data.append(temp)


In [ ]:
fields = ['id', 'n_liked_post', 'n_liked_comment', 'n_follower', 'n_following', 'account_privacy', 'switch_to_public', 'switch_to_private']
with open("./instagram_account_variable_data.csv", 'w') as f:
    w = csv.writer(f)

    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(saved_data)
print(len(saved_data))

In [ ]:
saved_data = []
for i in lines:

    p_likedposts = pth + "instagram_data/" + i[:-1] + "/likes/liked_posts.json"
    p_likedcomments = pth + "instagram_data/" + i[:-1] + "/likes/liked_comments.json"
    p_comments = pth + "instagram_data/" + i[:-1] + "/comments/post_comments.json"
    p_posts = pth + "instagram_data/" + i[:-1] + "/content/posts_1.json"

    print(i)
    if not os.path.exists(Path(p_posts)):
        continue
    temp = [i[:-1]]
    if not os.path.exists(Path(p_likedposts)):
        temp.append(0)
    else:
        with open(p_likedposts, 'r') as f:
            data = json.loads(f.read())
        data = pd.json_normalize(data, record_path =['likes_media_likes'])
        timelist = []
        for ind, r in data.iterrows():
            timelist.append(int(r[2][0]['timestamp']))
        freq = 0
        for ind in range(len(timelist)-1):
            freq += timelist[ind] - timelist[ind+1]
        freq = freq / len(timelist)-1
        temp.append(freq)

    if not os.path.exists(Path(p_likedcomments)):
        temp.append(0)
    else:
        with open(p_likedcomments, 'r') as f:
            data = json.loads(f.read())
        data = pd.json_normalize(data, record_path =['likes_comment_likes'])
        timelist = []
        for ind, r in data.iterrows():
            timelist.append(int(r[2][0]['timestamp']))
        freq = 0
        for ind in range(len(timelist)-1):
            freq += timelist[ind] - timelist[ind+1]
        freq = freq / len(timelist)-1
        temp.append(freq)

    if not os.path.exists(Path(p_comments)):
        temp.append(0)
    else:
        with open(p_comments, 'r') as f:
            data = json.loads(f.read())
        data = pd.json_normalize(data, record_path =['comments_media_comments'])
        timelist = []
        for ind, r in data.iterrows():
            timelist.append(int(r[2][0]['timestamp']))
        freq = 0
        for ind in range(len(timelist)-1):
            freq += timelist[ind] - timelist[ind+1]
        freq = freq / len(timelist)-1
        temp.append(freq)

    if not os.path.exists(Path(p_posts)):
        temp.append(0)
    else:
        with open(p_posts, 'r') as f:
            data = json.loads(f.read())
        timelist = []
        if isinstance(data, list):
            for r in data:
                timelist.append(int(r['media'][0]['creation_timestamp']))
            freq = 0
            for ind in range(len(timelist)-1):
                freq += timelist[ind] - timelist[ind+1]
            freq = freq / len(timelist)-1
        else:
            freq = data['creation_timestamp']
        temp.append(freq)

    saved_data.append(temp)

In [ ]:
import csv
fields = ['id', 'f_liked_post', 'f_liked_comment', 'f_comments', 'f_posts']
with open("./instagram_account_activity_data.csv", 'w') as f:
    w = csv.writer(f)

    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(saved_data)
print(len(saved_data))

In [ ]:
import os
import uuid
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials

KEY = "1d55848363814cf5b7a6c77b6f57250a"

ENDPOINT = "https://lpfacapi.cognitiveservices.azure.com/"

IMAGE_BASE_URL = 'https://csdx.blob.core.windows.net/resources/Face/Images/'

face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))

print('-----------------------------')
print()
print('DETECT FACES')
print()

from PIL import Image
#img = open('./252288878_271860704727291_8142963701425031159_n_17926857349794901.jpg', 'rb')


In [ ]:
"""Relavent features:
    "Number of faces detected: ", len(detected_faces)
    for face in detected_faces: 
        print (face.face_id)
        print (face.face_rectangle)
        #print (face.face_landmarks)
        print (face.face_attributes.blur)
        print (face.face_attributes.emotion)
        print (face.face_attributes.occlusion)
        print (face.face_attributes.head_pose)
        print (face.face_attributes.smile)
        print (face.face_attributes.quality_for_recognition)
    print()"""

def detection(filename):
    img = open(filename, 'rb')
    detected_faces = face_client.face.detect_with_stream(img, return_face_rectangle = True, return_face_landmarks=True, return_face_attributes=["headPose", "smile", "emotion", "occlusion", "blur", "qualityForRecognition"], detection_model='detection_01', recognition_model='recognition_04')
    print("fdjhakljfla", detected_faces)
    return detected_faces
    

In [ ]:
saved_data = []
n_face = []
k = 0

In [ ]:
m = k
#for i in lines:
for t in range(m, len(lines)):

    i = lines[t]
    p_profilephoto = pth + "instagram_data/" + i[:-1] + "/account_information/personal_information.json"

    if not os.path.exists(Path(p_profilephoto)):
        continue
    
    print(i)
    with open(p_profilephoto, 'r') as f:
            data = json.loads(f.read())
    if  'Profile Photo' in data['profile_user'][0]['media_map_data'].keys():
        p_profilephoto = pth + "instagram_data/" + i[:-1] + "/" + data['profile_user'][0]['media_map_data']['Profile Photo']['uri']
    elif not data['profile_user'][0]['media_map_data']:
        continue
    elif '\u00e5\u00a4\u00b4\u00e5\u0083\u008f' in data['profile_user'][0]['media_map_data'].keys():
        p_profilephoto = pth + "instagram_data/" + i[:-1] + '/' + data['profile_user'][0]['media_map_data']['\u00e5\u00a4\u00b4\u00e5\u0083\u008f']['uri']
    else:
        raise Exception('No face detected from image {}'.format(data['profile_user'][0]['media_map_data']))
    
    if not os.path.exists(Path(p_profilephoto)):
        print(p_profilephoto)
        continue

    detected_faces = detection(p_profilephoto)

    if not detected_faces:
        saved_data.append([i[:-1]]+[0]*21)
    else:
        for face in detected_faces:
            saved_data.append([i[:-1], face.face_rectangle.width, face.face_rectangle.height, 
                                face.face_rectangle.left, face.face_rectangle.top, face.face_attributes.blur.value, 
                                face.face_attributes.emotion.anger, face.face_attributes.emotion.contempt, face.face_attributes.emotion.disgust,
                                face.face_attributes.emotion.fear, face.face_attributes.emotion.happiness, face.face_attributes.emotion.neutral,
                                face.face_attributes.emotion.sadness, face.face_attributes.emotion.surprise, face.face_attributes.occlusion.forehead_occluded,
                                face.face_attributes.occlusion.eye_occluded, face.face_attributes.occlusion.mouth_occluded,
                                face.face_attributes.head_pose.roll, face.face_attributes.head_pose.yaw, face.face_attributes.head_pose.pitch,
                                face.face_attributes.smile, face.face_attributes.quality_for_recognition])
        n_face.append(len(detected_faces))
    k += 1
    

In [ ]:
import csv
fields = ['id', 'rec_width', 'rec_height', 'rec_left', 'rec_top', 'blur', 'emotion_anger', 'emotion_contempt', 'emotion_disgust', 'emotion_fear', 'emotion_happiness', 'emotion_neutral', 'emotion_sadness', 'emotion_surprise', 'forehead_occluded', 'eye_occluded', 'mouth_occluded', 'roll', 'yaw', 'pitch', 'smile', 'quality_for_recognition']
with open("./instagram_profile_photo_data.csv", 'w') as f:
    w = csv.writer(f)

    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(saved_data)
print(len(saved_data))

In [ ]:
print("Percent of face/no-face photo: ", len(n_face)/k)
print("Average number of faces: ", sum(n_face)/len(n_face))